<a href="https://colab.research.google.com/github/MichalSzulzyk/covid-death-analysis/blob/main/3_PredictApp_merge_scraping_list_of_hotels_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt -q update > /dev/null 2>&1 #Colab
!apt install -q chromium-chromedriver > /dev/null 2>&1 #Colab
!pip install -q selenium > /dev/null 2>&1 #Colab
!pip install -q webdriver-manager > /dev/null 2>&1 #Colab

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

import time

from datetime import datetime
from pytz import timezone

import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse, parse_qs, urljoin

import random

# TODO currency button (may be usefull in the future):
# next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[class*="a83ed08757"]')))
# next_button.click()

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--no-first-run")
chrome_options.add_argument("--disable-background-networking")
chrome_options.add_argument("--safebrowsing-disable-auto-update")
chrome_options.add_argument("--disable-sync")
chrome_options.add_argument("--metrics-recording-only")
chrome_options.add_argument("--disable-default-apps")

In [ ]:
# Get the current date in Warsaw time
warsaw_timezone = timezone('Europe/Warsaw')
current_time_warsaw = datetime.now(warsaw_timezone)
current_date_warsaw = current_time_warsaw.date()

In [ ]:
# @title Single locations html links (for testing)

#url = 'https://www.booking.com/hotel/pl/b-amp-b-poznan-old-town-poznan.pl.html'
#url = 'https://www.booking.com/hotel/pl/apartamenty-wodna-13-poznan.pl.html'
url = 'https://www.booking.com/hotel/pl/palazzo-rosso.pl.html'
#url = 'https://www.booking.com/hotel/pl/novotelpoznanmaltapoznan.pl.html'
#url = 'https://www.booking.com/hotel/pl/very-berry-apartamenty-nowe-jezyce.pl.html'

# Final code and functions (add booking prices to the Hotel List CSV):

In [ ]:
#@title Function to load and scrape availability data
def scrape_availability(url, num_clicks=2):
    # Initialize WebDriver
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    # Initialize wait
    wait = WebDriverWait(driver, 10)

    # Click on the date button
    date_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[data-testid="searchbox-layout-wide"] button[data-testid="date-display-field-start"]')))
    driver.execute_script("arguments[0].scrollIntoView();", date_button)
    actions = ActionChains(driver)
    actions.move_to_element(date_button).click().perform()

    # Wait for the calendar
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '[data-date]')))

    # Initialize an empty dictionary for date and price data
    date_price_dict = {}

    # Read initial dates and prices
    calendar_dates = driver.find_elements(By.CSS_SELECTOR, '[data-date]')
    read_dates_and_prices(calendar_dates, date_price_dict)

    # Load additional months and update the dictionary
    load_additional_months(num_clicks, driver, wait, date_price_dict)

    driver.quit()
    return date_price_dict



In [ ]:
#@title Function to load additional months
def load_additional_months(num_clicks, driver, wait, date_price_dict):
    next_button_xpath = "//button[.//span[@aria-hidden='true']/*[name()='svg']/*[name()='path'][contains(@d, 'M8.91289 19.2361')]]"
    for _ in range(num_clicks):
        # Find and click the button to load the next set of months
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
        next_button.click()
        time.sleep(1)  # Adjust the sleep time if necessary

        # Read new dates and prices and update the dictionary
        calendar_dates = driver.find_elements(By.CSS_SELECTOR, '[data-date]')
        read_dates_and_prices(calendar_dates, date_price_dict)


In [ ]:
#@title Function to read dates and prices in the cuurent calendar view

def read_dates_and_prices(calendar_dates, date_price_dict):
    for date_element in calendar_dates:
        date_str = date_element.get_attribute('data-date')
        date = datetime.strptime(date_str, '%Y-%m-%d').date()
        if date >= current_date_warsaw and date_str not in date_price_dict:
            price = 'Unavailable'
            try:
                price_element = WebDriverWait(date_element, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.b1f25950bd')))
                if price_element and price_element.text.strip() != '—':
                    price = price_element.text.strip()
            except TimeoutException:
                pass
            formatted_date = date.strftime('%Y-%m-%d')
            date_price_dict[formatted_date] = price


In [ ]:
#scrape_availability(url, num_clicks=2)

In [ ]:
folder_path = '/content/drive/My Drive/PredictApp' #Colab

# Specify the file name
file_name = 'df_hotels.csv'

# Combine folder path and file name to create the full file path
full_file_path = f'{folder_path}/{file_name}'

# Read the DataFrame
df_hotels = pd.read_csv(full_file_path)
df_hotels

df_hotels_test = df_hotels.head(2).copy()
df_hotels_test

,name,location,rating,html_link,address,availability
0,B&B Hotel Poznań Old Town,"Stare Miasto, Poznań","9,0",https://www.booking.com/hotel/pl/b-amp-b-pozna...,"16 plac Wolności, Stare Miasto, 61-738 Poznań,...","{'2023-11-11': '324', '2023-11-12': '229', '20..."
1,Apartamenty Strzelecka 34,"Stare Miasto, Poznań","9,3",https://www.booking.com/hotel/pl/apartamenty-s...,"34 Strzelecka, Stare Miasto, 61-846 Poznań, Po...","{'2023-11-11': 'Unavailable', '2023-11-12': '4..."


In [ ]:
# Initialize the 'availability' column with default values
df_hotels_test['availability'] = None

# Loop over each row in the DataFrame
for index, row in df_hotels_test.iterrows():
    # Apply the scrape_availability function to the 'html_link' column of the current row
    print(row['html_link'])
    availability_data = scrape_availability(row['html_link'])

    # Store the result in the 'availability' column
    df_hotels_test.at[index, 'availability'] = availability_data

    # time.sleep(random.randint(2, 7))
    time.sleep(3)

# Save the updated DataFrame
df_hotels_test.to_csv(full_file_path, index=False)

df_hotels_test


https://www.booking.com/hotel/pl/b-amp-b-poznan-old-town-poznan.pl.html
https://www.booking.com/hotel/pl/apartamenty-strzelecka-34.pl.html


,name,location,rating,html_link,address,availability
0,B&B Hotel Poznań Old Town,"Stare Miasto, Poznań","9,0",https://www.booking.com/hotel/pl/b-amp-b-pozna...,"16 plac Wolności, Stare Miasto, 61-738 Poznań,...","{'2023-11-13': '229', '2023-11-14': '229', '20..."
1,Apartamenty Strzelecka 34,"Stare Miasto, Poznań","9,3",https://www.booking.com/hotel/pl/apartamenty-s...,"34 Strzelecka, Stare Miasto, 61-846 Poznań, Po...","{'2023-11-13': '425', '2023-11-14': '425', '20..."


In [ ]:
df_hotels_test.loc[1, 'availability']

{'2023-11-13': '425',
 '2023-11-14': '425',
 '2023-11-15': '425',
 '2023-11-16': '425',
 '2023-11-17': '425',
 '2023-11-18': '425',
 '2023-11-19': '425',
 '2023-11-20': '425',
 '2023-11-21': '425',
 '2023-11-22': '425',
 '2023-11-23': '425',
 '2023-11-24': '425',
 '2023-11-25': '425',
 '2023-11-26': '425',
 '2023-11-27': '425',
 '2023-11-28': '425',
 '2023-11-29': '425',
 '2023-11-30': '425',
 '2023-12-01': '425',
 '2023-12-02': '425',
 '2023-12-03': '425',
 '2023-12-04': '425',
 '2023-12-05': '425',
 '2023-12-06': '425',
 '2023-12-07': '425',
 '2023-12-08': '425',
 '2023-12-09': '425',
 '2023-12-10': '425',
 '2023-12-11': '425',
 '2023-12-12': '425',
 '2023-12-13': '425',
 '2023-12-14': '425',
 '2023-12-15': '425',
 '2023-12-16': '425',
 '2023-12-17': '425',
 '2023-12-18': '425',
 '2023-12-19': '425',
 '2023-12-20': '425',
 '2023-12-21': '425',
 '2023-12-22': '425',
 '2023-12-23': 'Unavailable',
 '2023-12-24': 'Unavailable',
 '2023-12-25': '425',
 '2023-12-26': '425',
 '2023-12-27': '

In [ ]:
#TODO scraping: Opinie Gości (punktacja), Zdjęcia per pokój, ewentualnie per apartament, featury (Pralka etc...), czy śniadanie jest w cenie

Docelowo: Json: klucz ID hotelu (hash), pod tym kluczem data pobrania dancych i wartością słownik: Data - Wartość. codziennie robimy folder z nazwą danych i słownikiem.